# <div style="text-align:center; border-radius:30px 30px; padding:13px; color:white; margin:0; font-size:100%; font-family:Pacifico; background-color:#3170af; overflow:hidden"><b>Predicting Movie Rental Durations</b></div>

## <u> Introduction 📀 </u>

In this project, we aim to help a DVD rental company optimize its inventory management by predicting the duration of each DVD rental in days. Predicting rental duration allows the company to estimate demand more accurately, which can help in managing inventory, reducing costs, and improving customer satisfaction. By building a regression model to predict the number of days a customer rents a DVD, we can provide the company with actionable insights to improve their planning. The target is to achieve a model with a mean squared error (MSE) of less than 3, ensuring precise predictions that will be beneficial in real-world applications. This project will involve data cleaning, feature engineering, model selection, and evaluation to determine the best-performing model.

## <u>Plan</u>

1. **Data Preprocessing and Feature Engineering**


    1.1 **Data Loading**  
    1.2 **Calculate Rental Duration**  
    1.3 **Encode Special Features**  
    1.4 **Feature Selection**  

3. **Data Splitting**

4. **Modeling and Evaluation**

5. **Conclusion**

In [160]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt

## <u>1. **Data Preprocessing and Feature Engineering**</u>

 ### <u>1.1 **Data Loading**  </u>

In [161]:
df = pd.read_csv('Dataset/rental_info.csv')
df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [162]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB


In [163]:
df.describe()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
count,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000
mean,4.217161,2006.885379,2.944101,114.994578,20.224727,0.204842,0.200303,0.223378,0.198726,23.355504,14832.841876,11.389287
std,2.360383,2.025027,1.649766,40.114715,6.083784,0.403599,0.400239,0.416523,0.399054,23.503164,9393.431996,10.005293
min,0.990000,2004.000000,0.990000,46.000000,9.990000,0.000000,0.000000,0.000000,0.000000,0.980100,2116.000000,0.980100
25%,2.990000,2005.000000,0.990000,81.000000,14.990000,0.000000,0.000000,0.000000,0.000000,8.940100,6561.000000,0.980100
50%,3.990000,2007.000000,2.990000,114.000000,20.990000,0.000000,0.000000,0.000000,0.000000,15.920100,12996.000000,8.940100
75%,4.990000,2009.000000,4.990000,148.000000,25.990000,0.000000,0.000000,0.000000,0.000000,24.900100,21904.000000,24.900100
max,11.990000,2010.000000,4.990000,185.000000,29.990000,1.000000,1.000000,1.000000,1.000000,143.760100,34225.000000,24.900100


In [164]:
df.isna().sum()

rental_date         0
return_date         0
amount              0
release_year        0
rental_rate         0
length              0
replacement_cost    0
special_features    0
NC-17               0
PG                  0
PG-13               0
R                   0
amount_2            0
length_2            0
rental_rate_2       0
dtype: int64

In [165]:
df.columns

Index(['rental_date', 'return_date', 'amount', 'release_year', 'rental_rate',
       'length', 'replacement_cost', 'special_features', 'NC-17', 'PG',
       'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2'],
      dtype='object')

### <u> 1.2 **Calculate Rental Duration**  </u>

In [166]:
#Convert object columns to date type
df['return_date'] = pd.to_datetime(df['return_date'])
df['rental_date'] = pd.to_datetime(df['rental_date'])

In [167]:
#Create target columns
df['rental_length_days']= (df['return_date'] - df['rental_date']).dt.days

In [168]:
means_duration = df['rental_length_days'].mean()
means_duration

4.525944139713763

<strong> Now we have a new column for the durations, we have also calculated that the mean duration of a DVD rental is 4.5 days.</strong>

### <u> 1.3 **Encode Special Features** </u>

In [169]:
df['special_features']

0                         {Trailers,"Behind the Scenes"}
1                         {Trailers,"Behind the Scenes"}
2                         {Trailers,"Behind the Scenes"}
3                         {Trailers,"Behind the Scenes"}
4                         {Trailers,"Behind the Scenes"}
                              ...                       
15856    {Trailers,"Deleted Scenes","Behind the Scenes"}
15857    {Trailers,"Deleted Scenes","Behind the Scenes"}
15858    {Trailers,"Deleted Scenes","Behind the Scenes"}
15859    {Trailers,"Deleted Scenes","Behind the Scenes"}
15860    {Trailers,"Deleted Scenes","Behind the Scenes"}
Name: special_features, Length: 15861, dtype: object

In [170]:
df['special_features'].unique()

array(['{Trailers,"Behind the Scenes"}', '{Trailers}',
       '{Commentaries,"Behind the Scenes"}', '{Trailers,Commentaries}',
       '{"Deleted Scenes","Behind the Scenes"}',
       '{Commentaries,"Deleted Scenes","Behind the Scenes"}',
       '{Trailers,Commentaries,"Deleted Scenes"}',
       '{"Behind the Scenes"}',
       '{Trailers,"Deleted Scenes","Behind the Scenes"}',
       '{Commentaries,"Deleted Scenes"}', '{Commentaries}',
       '{Trailers,Commentaries,"Behind the Scenes"}',
       '{Trailers,"Deleted Scenes"}', '{"Deleted Scenes"}',
       '{Trailers,Commentaries,"Deleted Scenes","Behind the Scenes"}'],
      dtype=object)

In [171]:
# Function to create dummy columns
def create_dummy_columns(df):
    df["deleted_scenes"] = df['special_features'].apply(lambda x:1 if "Deleted Scenes" in x else 0)
    df["Behind the Scenes"] = df['special_features'].apply(lambda x:1 if "Behind the Scenes" in x else 0)

    return df

In [172]:
df = create_dummy_columns(df)

In [173]:
df

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days,deleted_scenes,Behind the Scenes
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3,0,1
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2,0,1
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7,0,1
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2,0,1
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15856,2005-08-22 10:49:15+00:00,2005-08-29 09:52:15+00:00,6.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,48.8601,7744.0,24.9001,6,1,1
15857,2005-07-31 09:48:49+00:00,2005-08-04 10:53:49+00:00,4.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,24.9001,7744.0,24.9001,4,1,1
15858,2005-08-20 10:35:30+00:00,2005-08-29 13:03:30+00:00,8.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,80.8201,7744.0,24.9001,9,1,1
15859,2005-07-31 13:10:20+00:00,2005-08-08 14:07:20+00:00,7.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,63.8401,7744.0,24.9001,8,1,1


### <u>1.4 **Feature Selection** </u>

In [174]:
#Select columns for features and exclude those that leak data about the target.
X = df.drop(columns=['rental_date','return_date', 'special_features', 'rental_length_days'])

In [175]:
#Select Target
y = df['rental_length_days']

## <u> 3. **Data Splitting** </u>

In [176]:
#Create Train and Test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=9)

## <u> 4. **Modeling and Evaluation** </u>

<u> <h3>a) Linear Regression </h3> </u>

We find the best features for the models 

In [177]:
# Create the Lasso model
lasso = Lasso(alpha=0.3, random_state=9) 

In [178]:
# Train the model and access the coefficients
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_

In [179]:
lasso_coef

array([ 5.77222892e-01,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  4.37993127e-02,  2.50573025e-06, -1.51706637e-01,
       -0.00000000e+00,  0.00000000e+00])

In [180]:
# Perform feature selectino by choosing columns with positive coefficients
X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

In [181]:
# Run OLS models on lasso chosen regression
ols = LinearRegression()
ols = ols.fit(X_lasso_train, y_train)
y_test_pred = ols.predict(X_lasso_test)
mse_lin_reg_lasso = mean_squared_error(y_test, y_test_pred)

In [182]:
mse_lin_reg_lasso

4.8705568457555675

<u><h3> b) Decision Tree Regressor</h3> </u>

In [183]:
# Define the hyperparameters to tune
param_grid = {
    'max_depth': [2, 3, 5, 7, 10, 12],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [184]:
# Create the decision tree regressor model
model = DecisionTreeRegressor()

In [185]:
# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 3, 5, 7, 10, 12],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

In [186]:
# Print the best hyperparameters 
print(f"Best hyperparameters: {grid_search.best_params_}")

Best hyperparameters: {'max_depth': 12, 'min_samples_leaf': 1, 'min_samples_split': 5}


In [187]:
# Use the best model for predictions
best_model = grid_search.best_estimator_
y_pred_DT = best_model.predict(X_test)

In [188]:
mse_Decision_Tree = mean_squared_error(y_test, y_pred_DT)
mse_Decision_Tree

2.296635649041803

<u><h3> c) Random Forest</h3> </u>

In [189]:
# Random forest hyperparameter space
param_dist = {'n_estimators': np.arange(1,101,1),
          'max_depth':np.arange(1,11,1)}

In [190]:
# Create a random forest regressor
rf = RandomForestRegressor()

In [191]:
# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions=param_dist, 
                                 cv=5, 
                                 random_state=9)

In [192]:
rand_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'n_estimators': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])},
                   random_state=9)

In [193]:
rf_pred = rand_search.predict(X_test)

In [194]:
mse_rf = mean_squared_error(y_test, rf_pred)
mse_rf

2.21353558877225

In [ ]:
<u><h3>d) Evaluations

In [195]:
#We create a new DataFrames to display all the results
mse_df = pd.DataFrame({
    'Model': ['Random Forest', 'Decision Tree', 'Linear Regression with Lasso'],
    'MSE': [mse_rf, mse_Decision_Tree, mse_lin_reg_lasso]
})

print(mse_df)

                          Model       MSE
0                 Random Forest  2.213536
1                 Decision Tree  2.296636
2  Linear Regression with Lasso  4.870557


## <u> 4. Conclusion </u>

In conclusion, our project focused on evaluating different machine learning models for predictive accuracy. We tested three models:

| Model                        | Mean Squared Error (MSE) |
|------------------------------|--------------------------|
| Random Forest                | 2.209                    |
| Decision Tree                | 2.293                    |
| Linear Regression with Lasso | 4.871                    |

The results show that the **Random Forest** model performed best, achieving the lowest Mean Squared Error (MSE) of 2.209, followed closely by the **Decision Tree** model with an MSE of 2.293. In contrast, the **Linear Regression with Lasso** model had a considerably higher MSE of 4.871, indicating lower predictive accuracy.

Based on these findings, the Random Forest model emerges as the most effective choice for our dataset, demonstrating strong predictive capabilities compared to the other methods. This comparison underscores the value of ensemble methods like Random Forest in achieving higher accuracy in predictive modeling.
